In [9]:
import os
import urllib2
from BeautifulSoup import BeautifulSoup
import datetime
import re
import pickle
import sys
from nltk.util import ngrams
import numpy as np
import string
import pymorphy2
import codecs

In [2]:
def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display
    
    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)
    
    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}
regex_punct = re.compile('[%s]' % re.escape(string.punctuation))
regex_spaces = re.compile(r'\s+', flags=re.UNICODE)

In [4]:
base_url = 'http://ria.ru/{}/more.html?id={}&date={}T{}'
# categories = ['politics', 'society', 'economy', 'incidents']
categories = ['politics', 'incidents']

site_urls = {}
id_ = '1447186104'

for category in log_progress(categories):
    tmpurls = []
    
    time_stamp = '000000'
#     date_stamp = str(datetime.date.today()).replace('-', '')
    date_stamp = '20160612'
    
    while len(tmpurls) < 200:
        req = urllib2.Request(base_url.format(category, id_, date_stamp, time_stamp), headers=hdr)
        page = urllib2.urlopen(req)
        soup = BeautifulSoup(page)

        articles = soup.findAll('div', attrs={'class': 'b-list__item'})

        for article in articles:
            tmpurl = 'http://www.ria.ru' + article.contents[0].attrs[0][1]
            tmpurls.append(tmpurl)
            title = article.findAll('span', attrs={'class': 'b-list__item-title'})[0].contents[0].contents[0]
#             break

        time_stamp = articles[-1].findAll('div',
                                         attrs={'class': 'b-list__item-time'})[0].contents[0].contents[0][:2]
        time_stamp = str(int(time_stamp) + 1) + '0000'
        date_stamp = articles[-1].findAll('div',
                                         attrs={'class': 'b-list__item-date'})[0].contents[0].contents[0]
        date_stamp = date_stamp.replace('.', '')
        date_stamp = date_stamp[4:] + date_stamp[2:4] + date_stamp[:2]
#         break
            
    site_urls[category] = tmpurls
    print '{}: {}'.format(category, len(tmpurls))
        
#     break

politics: 200
incidents: 200


In [5]:
class text_unit(object):
    
    def __init__(self, cat=None, title=None, text=None):
        self.category = cat
        self.title = title
        self.text = text

class corpus(object):
    
    def __init__(self, ent=[]):
        self.entities = []
        
    def add(self, entity):
        self.entities.append(entity)

    def get_by_cat(self, cat):
        return [el for el in self.entities if el.category == cat]
        
    @property
    def length(self):
        return len(self.entities)

In [6]:
corp = corpus()
morph = pymorphy2.MorphAnalyzer()

for cat, urls in site_urls.iteritems():
    counter = 0
    for url in log_progress(urls[:]):
#         if True:
        try:
            req = urllib2.Request(url, headers=hdr)
            page = urllib2.urlopen(req)
            soup = BeautifulSoup(page)

            soup_title = soup.findAll('h1', attrs={'class': 'b-article__title'})
            title = soup_title[0].contents[0].contents[0]

            text = ''
            soup_text = soup.findAll('div', attrs={'class': 'b-article__body js-mediator-article'})
            soup_text = soup_text[0].findAll('p')
            i = 0
            while len(soup_text[i].contents) < 2:
                i += 1
            text += soup_text[i].contents[-1].replace('&nbsp;',
                                                      ' ').replace('&mdash;',
                                                                   '-').strip().replace('&ndash;',
                                                                                        '-').strip()
#             print text
            for par in soup_text[i+1:]:
                text += par.contents[0].replace('&nbsp;',
                                                ' ').replace('&mdash;',
                                                             '-').strip().strip().replace('&ndash;',
                                                                                          '-').strip()
            text = regex_punct.sub(' ', text)
            text = regex_spaces.sub(' ', text)
            text = text.lower()
            
#             lemms = []
#             for word in text.split(' '):
#                 if len(word) > 2:
#                     norm = morph.parse(word)[0].normal_form
#                     if norm is not '':
#                         lemms.append(norm)
#             text = ' '.join(lemms)
#             print text
#             text += unicode(soup_text[0].contents[2].contents[1]).replace('&nbsp;',
#                                                                           ' ').replace('&mdash;',
#                                                                                        '-').strip()
#             print text
#             text += unicode(soup_text[0].contents[4].contents[0]).replace('&nbsp;',
#                                                                           ' ').replace('&mdash;',
#                                                                                        '-').strip()
#             print text

            tu = text_unit(cat, title, text)
            corp.add(tu)
#             break
        except:
#             print sys.exc_info()
            counter += 1
#             break
    print counter
#     break

102
93


In [7]:
print corp.length

205


---

In [13]:
f_3gramms = codecs.open('/home/alexandr/Desktop/3grams-3.txt', 'r', 'utf-8')
morph = pymorphy2.MorphAnalyzer()

base_3grams = {}
for line in f_3gramms:
#     print line
    line = regex_punct.sub(' ', line.strip())
#     print line
    line = regex_spaces.sub(' ', line)
#     print line
    tmp = line.split(' ')
    base_3grams[' '.join(tmp[1:])] = int(tmp[0])
#     break

In [14]:
len(base_3grams)

4504227

In [20]:
for cat in categories:
    print '{}:'.format(cat)
    texts = corp.get_by_cat(cat)
    ref_3grams = {}
    for text in texts:
        grmas = ngrams(text.text.split(' '), 3)
        for gram in grmas:
            if ref_3grams.get(' '.join(gram), None) is None:
                ref_3grams[' '.join(gram)] = 1
            else:
                ref_3grams[' '.join(gram)] += 1
#         break
    q1 = len(ref_3grams)
    
    tmp_sum = 0.0
    q2 = float(len(base_3grams))
    counter = 0
    for ngr, ref_freq in ref_3grams.iteritems():
#         print ngr
        base_freq = base_3grams.get(ngr, 0)
        if base_freq > 0:
            counter += 1
            tmp_sum -= np.log2(base_freq / q2)
#     perplexity = tmp_sum * (1 / float(q1))
    perplexity = tmp_sum * (1 / float(counter))
#         break
    print '\t3-grams match: {} of {}'.format(counter, q1)
    print '\tperplexity: {}'.format(perplexity)
    print

politics:
	3-grams match: 2139 of 12354
	perplexity: 18.0908995309

incidents:
	3-grams match: 1815 of 11766
	perplexity: 18.3436813846

